# RCS CNN+LSTM Pipeline (MetaTrader 5, Modular, Broker-Aware)

This notebook demonstrates a robust, modular pipeline for FX prediction using MetaTrader 5 data, advanced feature engineering, deep learning, and feature set analysis.

In [ ]:
# --- Data Loading ---
from async_data_loader import load_or_fetch, load_metatrader_data
import pandas as pd

symbols = ["EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCAD", "EURJPY", "GBPJPY"]
provider = "metatrader"
broker = "amp_global"
interval = "H1"

data = {}
for sym in symbols:
    df = load_or_fetch(
        symbol=sym,
        provider=provider,
        loader_func=load_metatrader_data,
        api_key="",
        interval=interval,
        broker=broker,
        force_refresh=False,
    )
    if "time" in df.columns and "close" in df.columns:
        df = df[["time", "open", "high", "low", "close", "tick_volume"]].dropna()
        df["time"] = pd.to_datetime(df["time"])
        df = df.set_index("time")
        for col in ["open", "high", "low", "close", "tick_volume"]:
            data[(sym, col)] = df[col]
prices = pd.DataFrame(data)
print(prices.head())

In [ ]:
# --- Feature Engineering ---
from feature_engineering import engineer_features
symbol = "EURUSD"
features = engineer_features(prices, symbol)
print(features.head())

In [ ]:
# --- Target Creation ---
target = (prices[(symbol, "close")].shift(-1) > prices[(symbol, "close")]).astype(int)
common_index = features.index.intersection(target.index)
features = features.loc[common_index]
target = target.loc[common_index]
print("features shape:", features.shape)
print("target shape:", target.shape)

In [ ]:
# --- Rolling Window, Train/Test Split, Model Training ---
from model_utils import create_rolling_windows, train_test_split_rolling, train_cnn_lstm_model, evaluate_cnn_lstm_model
from sklearn.preprocessing import StandardScaler

feature_names = features.columns.tolist()
features_scaled = StandardScaler().fit_transform(features)
lookback = 20
X = create_rolling_windows(features_scaled, lookback)
y = target.values[lookback:]
X_train, X_test, y_train, y_test = train_test_split_rolling(X, y, test_size=0.2)
model, history = train_cnn_lstm_model(X_train, y_train, X_test, y_test, X_train.shape[1:], epochs=10, batch_size=32)
acc, report = evaluate_cnn_lstm_model(model, X_test, y_test)
print("Test accuracy:", acc)
print(report)

In [ ]:
# --- Feature Set Comparison and Permutation Importance ---
from feature_set_analysis import compute_permutation_importance, compare_feature_sets, evaluate_and_save_feature_sets

manual_feature_sets = {
    "Indicators only": ['rsi', 'macd', 'momentum', 'cci'],
    "All features": feature_names
}
results_df, best_row = evaluate_and_save_feature_sets(features, y, manual_feature_sets, symbol, lookback_window=lookback, model_fn=None, epochs=10, batch_size=32)
print(results_df)

# Permutation importance for best feature set
best_features = best_row["Features"]
X_best = features[best_features].dropna().values
features_scaled_best = StandardScaler().fit_transform(X_best)
X_seq_best = create_rolling_windows(features_scaled_best, lookback)
y_seq_best = y[-len(X_seq_best):]
split = int(len(X_seq_best) * 0.8)
X_train_best, X_test_best = X_seq_best[:split], X_seq_best[split:]
y_train_best, y_test_best = y_seq_best[:split], y_seq_best[split:]
model_best, _ = train_cnn_lstm_model(X_train_best, y_train_best, X_test_best, y_test_best, X_train_best.shape[1:], epochs=10, batch_size=32)
importances = compute_permutation_importance(model_best, X_test_best, y_test_best, best_features)
importance_df = pd.DataFrame(importances, columns=["Feature", "Importance"])
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,5))
sns.barplot(data=importance_df, x="Importance", y="Feature")
plt.title(f"Feature Importance via Permutation: {symbol}")
plt.grid(True)
plt.show()